In [1]:
import numpy as np
import os

DATASETPATH = "/home/zantyr/.dataset"

model_path = "./models/2019-04-05T14:44:45.154857.h5"

test_source = np.load(os.path.join(DATASETPATH, "test_source.bin.npy"))
test_target = np.load(os.path.join(DATASETPATH, "test_target.bin.npy"))
test_phase  = np.load(os.path.join(DATASETPATH, "test_phase.bin.npy"))

In [2]:
import keras
import tensorflow as tf
model = keras.models.load_model(model_path, custom_objects={"tf": tf, "MAX": 26.937874})

Using TensorFlow backend.


Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


In [3]:
predictions = np.zeros(test_target.shape, np.float32)
for ix in range(test_source.shape[0]):
    print(ix, end=", ")
    predictions[ix] = model.predict(test_source[ix:ix+1])[0]

0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 

In [4]:
import scipy.io.wavfile as sio
import subprocess
import tempfile
import re

def pesq(gt, pred, phase):
    spec = (np.sqrt(np.exp(-gt)) * 512) * np.exp(phase * 1j)
    sound = np.zeros(spec.shape[0] * 128 + 512 - 128)
    for i in range(spec.shape[0]):
        frame = np.zeros(257, np.complex)
        frame[:257] = spec[i,:]
        frame = np.fft.irfft(frame)
        sound[128 * i : 128 * i + 512] += frame
    spec = (np.sqrt(np.exp(-pred)) * 512) * np.exp(phase[:, :129] * 1j)
    sound2 = np.zeros(spec.shape[0] * 128 + 512 - 128)
    for i in range(spec.shape[0]):
        frame = np.zeros(257, np.complex)
        frame[:129] = spec[i,:]
        frame = np.fft.irfft(frame)
        sound2[128 * i : 128 * i + 512] += frame
    fname_gt = tempfile.mktemp() + ".wav"
    fname_pred = tempfile.mktemp() + ".wav"
    sio.write(fname_gt, 16000, (2**15 * sound).astype(np.int16))
    sio.write(fname_pred, 16000, (2**15 * sound2).astype(np.int16))
    ot,e = subprocess.Popen(["PESQ", "+wb", "+16000", fname_gt, fname_pred], stdout = subprocess.PIPE, stderr = subprocess.PIPE).communicate()
    os.remove(fname_gt)
    os.remove(fname_pred)
    o = ot.decode("utf-8").split('\n')[-2]
    value = re.findall("= \d\.\d+", o)[0]
    return float(value[2:])

quality = []
for i in range(test_target.shape[0]):
    print(i)
    try:
        quality.append(pesq(test_target[i], test_source[i], test_phase[i]))
    except IndexError:
        print("Failed getting PESQ value for recording {}".format(i))
quality = np.array(quality)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [5]:
print("Predicted by model: ", quality.mean(), "+-", quality.std())

Predicted by model:  1.3868 +- 0.3262864692260468
